In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from time import time
from catboost import CatBoostClassifier
import gc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint
from hyperopt import hp, tpe, STATUS_OK, Trials, fmin
from sklearn.ensemble import GradientBoostingClassifier

start = time()
train = pd.read_csv('D:/Driver/ohe_train_v2.csv')
train.fillna(-1,inplace=True)
# test = pd.read_csv('D:/Driver/ohe_test_v2.csv',na_values=-1)

unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(unwanted, axis=1)
# test = test.drop(unwanted, axis=1)

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = train[features].values
y = train['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

def objective(space):
    clf = GradientBoostingClassifier(n_estimators=space['n_estimators'],
                                     subsample=space['subsample'],
                                     max_features=space['max_features'],
                                     max_depth=space['max_depth'],
                                     min_samples_split=space['min_samples_split'],
                                     min_samples_leaf=space['min_samples_leaf'],
                                     random_state=2017,
                                     verbose=5,
                                )

    clf.fit(X_train, y_train)

    pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, pred)
    return {'loss': 1-auc, 'status': STATUS_OK}

#     skf = StratifiedKFold(n_splits=5, random_state=2017)
#     scores = cross_val_score(clf, X, y,
#                              cv=skf, scoring='roc_auc', n_jobs=-1)
#     return {'loss': 1 - scores.mean(), 'status': STATUS_OK}


space = {
    'n_estimators': hp.choice('n_estimators', np.arange(200, 1100, 100, dtype=int)),
    'max_depth': hp.choice('max_depth', np.arange(3, 16, dtype=int)),
    'max_features': hp.quniform('max_features', 0.4, 1.0, 0.05),
    'min_samples_split': hp.choice('min_samples_split', np.arange(2, 501, 1)),
    'min_samples_leaf': hp.choice('min_samples_leaf', np.arange(1, 501, 1)),
    'subsample': hp.quniform('subsample', 0.6, 1.0, 0.05),
}

trials = Trials()
best = fmin(objective,
            space,
            algo=tpe.suggest,
            max_evals=1,  # change
            trials=trials)

print(best)
print('Time: {} mins'.format((time() - start) / 60))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.3084           0.0012          188.24m
         2           0.3128           0.0010          208.54m
         3           0.3115           0.0008          209.10m
         4           0.3081           0.0006          206.40m
         5           0.3079           0.0006          208.55m
         6           0.3073           0.0004          208.59m
         7           0.3071           0.0004          205.84m
         8           0.3063           0.0003          206.68m
         9           0.3033           0.0003          206.29m
        10           0.3035           0.0003          206.43m
        11           0.3054           0.0002          205.76m
        12           0.3034           0.0002          205.69m
        13           0.3042           0.0002          206.01m
        14           0.3042           0.0002          206.94m
        15           0.3067           0.0001          205.27m
       

KeyboardInterrupt: 